# Linear Regression Modeling


Now that I have a problem statement in hand, I can begin the modeling process. I will be creating a linear model that explores the relationships between binge drinking, sex/gender, and mortality rates by self-harm.

## Problem Statement
As the proportion of the population that binge drinks increases, does the mortality rate by self harm change? 



---
### Process

In this notebook, I'll investigate relationships between self-harm mortality rate and other variables, such as alcohol use/type, state, and sex, as well as between unemployment rate and labor force.

The models in this notebook will all be simple univariate linear models that directly compare alcohol prevalence use, sex, state, unemployment or labor force with self-harm mortality. 


In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# imports - modeling
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest

#sklearn.set_config(display='diagram')

In [2]:
# read in data
# 'labor_force': 'Int64', 'employed': 'float64', 'unemployed': 'float64', 'unemployment_rate': 'float64'

#read in joined selfharm_alcohol use data
harm_alcohol_any = pd.read_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_alcohol_any.csv', dtype={'FIPS': 'object'})
harm_alcohol_heavy = pd.read_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_heavy_prop_heavy.csv', dtype={'FIPS': 'object'})
harm_alcohol_binge = pd.read_csv('../data/cleaned/selfharm_alcohol_joins/selfharm_join_binge_prop_binge.csv', dtype={'FIPS': 'object'})

#drop the NAs
harm_alcohol_any.dropna(inplace=True)
harm_alcohol_heavy.dropna(inplace=True)
harm_alcohol_binge.dropna(inplace=True)


### First - Establishing a Baseline
To get a baseline accuracy score

In [56]:
harm_alcohol_binge['mx'].mean()

15.635809937644858

In [12]:
harm_alcohol_any

,location_name,FIPS,cause_name,sex,year_id,mx,state,UID,UID_2,county_state_y,alcohol_any
22,Adams County,17001,Self-harm,Male,2002,19.723610,Illinois,17001-2002,17001-Male-2002,"Adams County, Illinois",64.2
23,Adams County,17001,Self-harm,Male,2003,19.560579,Illinois,17001-2003,17001-Male-2003,"Adams County, Illinois",65.3
24,Adams County,17001,Self-harm,Male,2004,19.175775,Illinois,17001-2004,17001-Male-2004,"Adams County, Illinois",64.0
25,Adams County,17001,Self-harm,Male,2005,20.508569,Illinois,17001-2005,17001-Male-2005,"Adams County, Illinois",64.5
26,Adams County,17001,Self-harm,Male,2006,19.711650,Illinois,17001-2006,17001-Male-2006,"Adams County, Illinois",64.8
...,...,...,...,...,...,...,...,...,...,...,...
331058,Wyoming County,54109,Self-harm,Both,2008,21.159870,West Virginia,54109-2008,54109-Both-2008,"Wyoming County, West Virginia",23.5
331059,Wyoming County,54109,Self-harm,Both,2009,21.284966,West Virginia,54109-2009,54109-Both-2009,"Wyoming County, West Virginia",22.6
331060,Wyoming County,54109,Self-harm,Both,2010,21.706674,West Virginia,54109-2010,54109-Both-2010,"Wyoming County, West Virginia",23.1
331061,Wyoming County,54109,Self-harm,Both,2011,22.570620,West Virginia,54109-2011,54109-Both-2011,"Wyoming County, West Virginia",23.3


# Model 1 - Simple Univariate Regression Models
Here, I will not take into account state, county, sex or year. I will simply see if there appears to be a relationship between the different types of alcohol use/prevalence and `mx` (self-harm mortality rate). 

My success metrics will be accuracy and RMSE.

I will write a function that will allow me to try several different regression models with less code. 

In [3]:
# instantiate different regression models and put them in a list
lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()
rf = RandomForestRegressor()
ada = AdaBoostRegressor()
gboost = GradientBoostingRegressor()

estimators = [lr, lasso, ridge, rf, ada, gboost]

In [7]:
# function
def model_eval(df, independent_var, dependent_var, estimator_list):
    
    #drop the "both" sexes rows, so we're left with only male and female
    df = df[df['sex'] != 'Both']
    
    X = df[[independent_var]]
    y = df[dependent_var]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    for estimator in estimator_list:
        estimator.fit(X_train, y_train)

        train_score = round(estimator.score(X_train, y_train), 2)
        test_score = round(estimator.score(X_test, y_test), 2)

        rmse = round(mean_squared_error(y_test, estimator.predict(X_test), squared=False), 2)
        
        print(f'Estimator: {estimator}, Train/Test Accuracy: {train_score, test_score}, RMSE: {rmse}')
    
    return None
    

In [8]:
# regressions for alcohol_any
model_eval(harm_alcohol_any, 'alcohol_any', 'mx', estimators)

Estimator: LinearRegression(), Train/Test Accuracy: (0.15, 0.15), RMSE: 10.03
Estimator: Lasso(), Train/Test Accuracy: (0.15, 0.15), RMSE: 10.03
Estimator: Ridge(), Train/Test Accuracy: (0.15, 0.15), RMSE: 10.03
Estimator: RandomForestRegressor(), Train/Test Accuracy: (0.17, 0.15), RMSE: 10.05
Estimator: AdaBoostRegressor(), Train/Test Accuracy: (0.14, 0.14), RMSE: 10.12
Estimator: GradientBoostingRegressor(), Train/Test Accuracy: (0.16, 0.16), RMSE: 9.98


In [9]:
# regressions for alcohol_heavy
model_eval(harm_alcohol_heavy, 'alcohol_heavy', 'mx', estimators)

Estimator: LinearRegression(), Train/Test Accuracy: (0.39, 0.38), RMSE: 8.6
Estimator: Lasso(), Train/Test Accuracy: (0.38, 0.38), RMSE: 8.6
Estimator: Ridge(), Train/Test Accuracy: (0.39, 0.38), RMSE: 8.6
Estimator: RandomForestRegressor(), Train/Test Accuracy: (0.43, 0.41), RMSE: 8.37
Estimator: AdaBoostRegressor(), Train/Test Accuracy: (0.31, 0.3), RMSE: 9.17
Estimator: GradientBoostingRegressor(), Train/Test Accuracy: (0.43, 0.42), RMSE: 8.32


In [10]:
# regressions for alcohol_prop_heavy
model_eval(harm_alcohol_heavy, 'alcohol_prop_heavy', 'mx', estimators)

Estimator: LinearRegression(), Train/Test Accuracy: (0.49, 0.47), RMSE: 7.96
Estimator: Lasso(), Train/Test Accuracy: (0.49, 0.47), RMSE: 7.96
Estimator: Ridge(), Train/Test Accuracy: (0.49, 0.47), RMSE: 7.96
Estimator: RandomForestRegressor(), Train/Test Accuracy: (0.56, 0.52), RMSE: 7.53
Estimator: AdaBoostRegressor(), Train/Test Accuracy: (0.44, 0.42), RMSE: 8.31
Estimator: GradientBoostingRegressor(), Train/Test Accuracy: (0.56, 0.53), RMSE: 7.49


In [11]:
# regressions for alcohol_binge
model_eval(harm_alcohol_binge, 'alcohol_binge', 'mx', estimators)

Estimator: LinearRegression(), Train/Test Accuracy: (0.43, 0.44), RMSE: 8.15
Estimator: Lasso(), Train/Test Accuracy: (0.43, 0.44), RMSE: 8.15
Estimator: Ridge(), Train/Test Accuracy: (0.43, 0.44), RMSE: 8.15
Estimator: RandomForestRegressor(), Train/Test Accuracy: (0.52, 0.51), RMSE: 7.61
Estimator: AdaBoostRegressor(), Train/Test Accuracy: (0.21, 0.22), RMSE: 9.64
Estimator: GradientBoostingRegressor(), Train/Test Accuracy: (0.52, 0.51), RMSE: 7.58


In [12]:
# regressions for alcohol_binge
model_eval(harm_alcohol_binge, 'alcohol_prop_binge', 'mx', estimators)

Estimator: LinearRegression(), Train/Test Accuracy: (0.64, 0.64), RMSE: 6.53
Estimator: Lasso(), Train/Test Accuracy: (0.64, 0.64), RMSE: 6.53
Estimator: Ridge(), Train/Test Accuracy: (0.64, 0.64), RMSE: 6.53
Estimator: RandomForestRegressor(), Train/Test Accuracy: (0.74, 0.72), RMSE: 5.72
Estimator: AdaBoostRegressor(), Train/Test Accuracy: (0.66, 0.66), RMSE: 6.39
Estimator: GradientBoostingRegressor(), Train/Test Accuracy: (0.74, 0.73), RMSE: 5.67


### Simple Univariate Regression Model Summary

There doesn't appear to be much of a substantial relationship between `alcohol_any` and the self harm mortality rate, so we'll scrap that variable moving forward. 

None of the model/input variable combinations appear to show overfitting, so we can focus solely on which variables / models to include moving forward. 

`alcohol_prop_heavy` and `alcohol_prop_binge` appear to produce models with the strongest accuracy scores (although `alcohol_binge` only just barely underperforms `alcohol_prop_heavy` on measures of accuracy (except on the AdaBoostRegressor). 

Our best scoring model/variable combo used `alcohol_prop_binge` with RandomForestRegressor and the GradientBoostRegressor producing nearly identical accuracy and RMSE results. 

---

## Model 1 Optimization: Fine Tuning Our Best Simple Regression Models

Since we identified that the RandomForestRegressor and GradientBoostRegressor performed *exceptionally* well in terms of a accuracy, we will now hypertune the parameters to see if we can drive the accuracy up even higher. 


In [14]:
### GridSearching for params
harm_alcohol_binge_both = harm_alcohol_binge[harm_alcohol_binge['sex'] != 'Both']

X = harm_alcohol_binge_both[['alcohol_prop_binge']]
y = harm_alcohol_binge_both['mx']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
rf = RandomForestRegressor(n_jobs=-1)
rf_params = {'criterion': ['squared_error', 'poisson'], 'max_features': ['auto', 'sqrt', 'log2'], 'verbose': [True, False]}

gboost = GradientBoostingRegressor()
gb_params = {'loss': ['squared_error', 'absolute_error', 'huber'], 'n_estimators': [100, 200], 'max_depth': [3, 5, 10]}

In [16]:
# write a grid search function that returns the train, test, rmse and best_params
def grid_search_eval(est, params):
    #est = estimator(n_jobs = -1)
    # instantiate model
    gs = GridSearchCV(estimator=est, param_grid=params)
    
    # fit the model
    gs.fit(X_train, y_train)
    
    # print out metrics
    print("Train Score: ", gs.score(X_train, y_train))
    print('Test Score: ', gs.score(X_test, y_test))
    print("RMSE: ", mean_squared_error(y_test, gs.predict(X_test), squared=False))
    print('Best Params: ', gs.best_params_)
    
    #return the gs instance in case we want to work with it
    return gs
    
    

In [17]:
# grid search for random forest
grid_search_eval(rf, rf_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n

KeyboardInterrupt: 

In [ ]:
grid_search_eval(gboost, gb_params)

In [51]:
lr.coef_, lr.intercept_

(array([0.82007253]), -11.746317231263697)

### Model 2 Summary: 


### Random Forest Regression with lots of vars